# Create CV dataset

This notebook creates a dataset that enables 3-fold cross-validation for the NNet bot. 

In [3]:
import draftsimtools as ds

### Initial dataset
Load data from original M19 draftsim data files and process it. Some of this processing is unnecessary since it was already performed, but better safe than sorry

In [6]:
dataset_path = "../../data/"
rating_path1 = dataset_path + "m19_rating.tsv"
rating_path2 = dataset_path + "m19_land_rating.tsv"
drafts_path = dataset_path + "full_dataset/train.csv"
cur_set = ds.create_set(rating_path1, rating_path2)
raw_drafts = ds.load_drafts(drafts_path)
cur_set, raw_drafts = ds.fix_commas(cur_set, raw_drafts)
le = ds.create_le(cur_set["Name"].values)
drafts = ds.process_drafts(raw_drafts)
drafts = [d for d in drafts if len(d)==45] # Remove imconplete drafts

Processing draft: 0.
Processing draft: 10000.
Processing draft: 20000.
Processing draft: 30000.
Processing draft: 40000.
Processing draft: 50000.
Processing draft: 60000.
Processing draft: 70000.
Processing draft: 80000.


### Split data

Splits data for 3-fold cross-validation and saves to file.

In [12]:
third = int(len(drafts) / 3)
fold1 = drafts[0:third]
fold2 = drafts[third:(third*2)]
fold3 = drafts[(third*2):(len(drafts) + 1)]
print(len(fold1))
print(len(fold2))
print(len(fold3))
print(len(fold1) + len(fold2) + len(fold3) == 86359)

28786
28786
28787
True
